# Analysis of DNA binding mutants 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pystan
import glob
import bokeh.io
import bokeh.plotting
import bokeh.palettes
import sys
sys.path.insert(0, '../../')
import mut.bayes
import mut.viz
import mut.stats
import mut.thermo
bokeh.io.output_notebook()
colors = mut.viz.pub_style()
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading BokehJS ...

In [2]:
# Load all of the 2018 flow data.
flow_files = glob.glob('../processing/2018*flow*/output/*fold_change.csv')
dfs = [pd.read_csv(f, comment='#') for f in flow_files]
flow_data = pd.concat(dfs, axis=0)
flow_data = flow_data[(flow_data['fold_change'] >= -0.2) & (flow_data['fold_change'] <= 1.2)]

# Load the microscopy data
mic_files = glob.glob('../processing/2018*microscopy*/output/*fold_change.csv')
dfs = [pd.read_csv(f) for f in mic_files]
mic_data = pd.concat(dfs, axis=0)

In [3]:
# Isolate the leakiness data
leakiness = flow_data[(flow_data['IPTGuM'] ==0) & (flow_data['mutant'] != 'Q21M') &
                     (flow_data['mutant'] != 'wt') & (flow_data['mutant'] != 'auto') 
                     & (flow_data['mutant'] != 'delta') & (flow_data['mutant'] != np.nan) &
                     (flow_data['mutant'] != 'Q294K') & (flow_data['mutant'] != 'Q294V') &
                     (flow_data['mutant'] != 'F164T')].copy()
leakiness.drop('Unnamed: 0', axis=1, inplace=True)
leakiness.rename(columns={'mean_FITC_H':'mean_intensity'}, inplace=True)
leakiness.loc[:, 'method'] = 'flow'
mic_data.loc[:, 'method'] = 'microscopy'
merged = pd.concat([leakiness, mic_data], ignore_index=True)
merged = merged[(merged['mutant'] != 'none')]

In [4]:
# Plot the data points. 
p1 = bokeh.plotting.figure(x_axis_label='repressor copy number',
                          y_axis_label='fold-change',
                          x_axis_type='log', y_axis_type='log',
                          plot_width=600, plot_height=400)

grouped = merged.groupby(['mutant'])
colors = bokeh.palettes.Accent8
i = 0
for g, d in grouped:
    _grouped = d.groupby('repressors')
    for _g, _d in _grouped:
        _d = _d.mean()
        p1.circle(_d['repressors'], _d['fold_change'], legend=g,
                     color=colors[i])
    i += 1
bokeh.io.show(p1)


In [5]:
# Include IDs for heirarchical model.
merged.loc[:, 'id'] = 1
merged.loc[merged['mutant'] == 'Q21A', 'id'] = 2
merged.loc[merged['mutant'] == 'Q21M', 'id'] = 3

In [6]:
merged.head()

,IPTGuM,date,fold_change,mean_YFP,mean_intensity,method,mutant,operator,repressors,run_number,strain,username,id
0,0.0,20180322,0.761336,NaN,12694.551352,flow,Q21A,O2,60.0,NaN,R60,gchure,2
1,0.0,20180322,0.279661,NaN,9191.802448,flow,Q21A,O2,260.0,NaN,R260,gchure,2
2,0.0,20180322,0.632680,NaN,11758.961989,flow,Q21A,O2,124.0,NaN,R124,gchure,2
3,0.0,20180322,0.155053,NaN,8285.651273,flow,Q21A,O2,1220.0,NaN,R1220,gchure,2
4,0.0,20180123,0.197116,NaN,5387.374666,flow,Q21A,O2,260.0,NaN,R260,gchure,2


In [7]:
def assemble_StanModelCode(model_file, function_file):
    """Returns a string of the stan model code from a model and function file"""
    lines = []
    files = [function_file, model_file]
    for f in files:
        with open(f, 'r') as file:
            out = file.read().splitlines()
            for line in out:
                lines.append(line) 
    model_code = """\n"""
    for line in lines:
        model_code += line + '\n'
    return model_code

In [8]:
model_code = assemble_StanModelCode('../stan/heirarchical_epR_fit.stan', '../stan/functions.stan')
sm = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_31d86c808853debbc6ac3a765ef665db NOW.


In file included from /var/folders/2q/lvh2zsws3lxckq8xtkn_84z80000gn/T/tmpt9blc56v/stanfit4anon_model_31d86c808853debbc6ac3a765ef665db_2886748525508683860.cpp:599:
In file included from /anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18:
In file included from /anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1816:
/anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: "Using deprecated NumPy API, disable it by "          "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it by " \
 ^
/var/folders/2q/lvh2zsws3lxckq8xtkn_84z80000gn/T/tmpt9blc56v/stanfit4anon_model_31d86c808853debbc6ac3a765ef665db_2886748525508683860.cpp:9155:30: warning: comparison of integers of different signs: 'Py_ssize_t' (aka 'long') and 'std::_

In [9]:
merged = merged[(merged['fold_change'] >= -0.2) & (merged['fold_change'] <= 1.3)]
# Assemble the data dictionary. 
ka = 139
ka_sigma = 30
ki = 0.53
ki_sigma = 0.04
data_dict = {'J':3, 'N':len(merged), 'trial':merged['id'].values,
            'R':merged['repressors'].values, 'n_ns':4.6E6, 'ka':ka, 'ki':ki,
            'ep_AI':4.5, 'n_sites':2, 'c':np.zeros(len(merged)), 'fc':merged['fold_change'].values}

# Sample the distribution.
samples = sm.sampling(data=data_dict, iter=5000, chains=4)

/anaconda3/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [17]:
def chains_to_dataframe(fit):
    data = fit.extract()
    keys = list(data.keys())
    varnames = [k for k in keys if 'lp__' not in k]
    samples = {}
    for i, key in enumerate(varnames):
        # Get the shape.
        dim = np.shape(data[key])
        if len(dim) == 2:
            for j in range(dim[-1]):
                samples['{}.{}'.format(key, j+1)] = data[key][:, j]
    
        else:
            samples[key] = data[key]
            
    # compute the log_post. 
    new_keys = samples.keys()
    logp = []
    for j in range(dim[0]):
        logp.append(fit.log_prob([samples[k][j] for k in fit.unconstrained_param_names()]))
        
    samples['logp'] = logp
    return pd.DataFrame(samples)
    

In [18]:
df = chains_to_dataframe(samples)

df.rename(columns={'ep_R.1':'epR_Y20I', 'ep_R.2':'epR_Q21A', 'ep_R.3':'epR_Q21M'},
         inplace=True)

In [19]:
stats = mut.stats.compute_statistics(df)

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [20]:
stats

,parameter,mode,hpd_min,hpd_max
0,epR_Y20I,-9.575945,-9.934566,-9.442711
1,epR_Q21A,-10.327062,-10.472704,-10.096274
2,epR_Q21M,-15.081613,-15.250043,-14.995302
3,sigma.1,0.091401,0.084407,0.157583
4,sigma.2,0.051097,0.066326,0.125643
5,sigma.3,0.001473,0.000943,0.003306


In [21]:
# Plot the fits. 
rep_range = np.logspace(0, 4, 500)
muts = merged['mutant'].unique()
ep_R = [stats[stats['parameter']=='epR_{}'.format(m)]['mode'].values[0] for m in muts]
ep_R_low = [stats[stats['parameter']=='epR_{}'.format(m)]['hpd_min'].values[0] for m in muts]
ep_R_high = [stats[stats['parameter']=='epR_{}'.format(m)]['hpd_max'].values[0] for m in muts]
meshed_R, meshed_ep = np.meshgrid(rep_range, ep_R)
meshed_R_low, meshed_ep_high = np.meshgrid(rep_range, ep_R_high)
meshed_R_high, meshed_ep_low = np.meshgrid(rep_range, ep_R_low)
arch = mut.thermo.SimpleRepression(R=meshed_R, ep_r=meshed_ep, ep_ai=4.5, ka=139, ki=0.53,
                                  n_sites=int(2), n_ns=4.6E6, effector_conc=0)
arch_high = mut.thermo.SimpleRepression(R=meshed_R_high, ep_r=meshed_ep_high, ep_ai=4.5, ka=139, ki=0.53,
                                  n_sites=int(2), n_ns=4.6E6, effector_conc=0)
arch_low = mut.thermo.SimpleRepression(R=meshed_R_low, ep_r=meshed_ep_low, ep_ai=4.5, ka=139, ki=0.53,
                                  n_sites=int(2), n_ns=4.6E6, effector_conc=0)
fc = arch.fold_change()
fc_high = arch_high.fold_change()
fc_low = arch_low.fold_change()
# Set up the figure. 
color_cycle = ['tomato', 'dodgerblue', 'slategrey']
color_dict = {i: j for i, j in zip(muts, color_cycle)}
p = bokeh.plotting.figure(plot_width=600, plot_height=400, x_axis_type='log',
                         y_axis_type='log', x_axis_label='repressor copy number',
                         y_axis_label='fold-change')
for i in range(3):
    p.line(rep_range, fc[i, :], color=color_cycle[i], line_width=2, 
           legend='{} = {:0.1f} +{:0.1f} -{:0.1f} k_BT'.format(muts[i], ep_R[i], 
                    np.abs(ep_R[i]) - np.abs(ep_R_high[i]), 
                    np.abs(ep_R_low[i]) - np.abs(ep_R[i])), alpha=0.5)
    band_x = np.append(rep_range, rep_range[::-1])
    band_y = np.append(fc_low[i, ], fc_high[i, ::-1])
    p.patch(band_x, band_y, color=color_cycle[i], alpha=0.3)
    
    
# Plot the data points. 
grouped = merged.groupby(['mutant', 'repressors'])
for g, d in grouped:
    p.circle(g[1], d['fold_change'].mean(), color=color_dict[g[0]])
p.legend.location = 'bottom_left'    
bokeh.io.show(p)

###  Plotting the titration curves.

In [22]:
c_range = np.logspace(-2, 4, 500)
reps = flow_data['repressors'].unique()
reps = np.sort(reps[reps > 0])

# 
c, ep, R = np.meshgrid(c_range, ep_R, reps) 
c_low, ep_low, R_low = np.meshgrid(c_range, ep_R_low, reps) 
c_high, ep_high, R_high = np.meshgrid(c_range, ep_R_high, reps) 
fc = mut.thermo.SimpleRepression(R=R, ep_r=ep, effector_conc=c, ep_ai=4.5,
                                ka=ka, ki=ki, n_sites=2, n_ns=4.6E6).fold_change()
fc_low = mut.thermo.SimpleRepression(R=R_low, ep_r=ep_low, effector_conc=c_low, ep_ai=4.5,
                                ka=ka, ki=ki, n_sites=2, n_ns=4.6E6).fold_change()   
fc_high = mut.thermo.SimpleRepression(R=R_high, ep_r=ep_high, effector_conc=c_high, ep_ai=4.5,
                                ka=ka, ki=ki, n_sites=2, n_ns=4.6E6).fold_change()

fcs = [fc, fc_low, fc_high]

rep_color = bokeh.palettes.Category10_5
rep_dict = {i:j for i, j in zip(reps, rep_color)}
canvas = []
for i, m in enumerate(muts):
    _p = bokeh.plotting.figure(plot_width=350, plot_height=250, x_axis_type='log',
                               x_axis_label='IPTG [µM]', y_axis_label='fold-change',
                              title=m)
    for j, R in enumerate(reps):
        if i == 0:
            legend = str(R)
        else:
            legend = False
        _p.line(c_range, fc[i, :, j], color=rep_dict[R], line_width=1, alpha=0.5,
               legend=legend)
        band_x = np.append(c_range, c_range[::-1])
        band_y = np.append(fc_low[i,:, j], fc_high[i, ::-1, j])
        _p.patch(band_x, band_y, color=rep_dict[R], alpha=0.3)
   
        
        # Plot the data.
        mut_flow = flow_data[(flow_data['mutant'] == m) & (flow_data['repressors'] == R)]
        grouped = mut_flow.groupby('IPTGuM')
        for g, d in grouped:
            mean_fc = d['fold_change'].mean()
            sem_fc = d['fold_change'].std() / np.sqrt(len(d))
            
            _p.circle(g, mean_fc, color=rep_dict[R], size=2)
            _p.line((g, g), (mean_fc - sem_fc, mean_fc + sem_fc), color=rep_dict[R])
            
        
    if i==0:
        _p.legend.location = 'bottom_right'
    canvas.append(_p)
    
layout = bokeh.layouts.gridplot([[canvas[0], canvas[1]], [canvas[2]]])
bokeh.io.show(layout)

In [26]:
import imp
imp.reload(mut.thermo)

<module 'mut.thermo' from '../../mut/thermo.py'>

##  Computing the properties.

In [44]:
# assemble the data ignoring Q21M
dna_muts = flow_data[(flow_data['mutant'] == 'Q21M') | (flow_data['mutant']=='Y20I') | 
                    (flow_data['mutant'] == 'Q21A')]
dna_muts = dna_muts[(dna_muts['mutant'] != 'Q21M') | (dna_muts['IPTGuM'] > 0)]
mic_data['IPTGuM'] = 0

merged_global = pd.concat([mic_data[mic_data['mutant']=='Q21M'], dna_muts])

# Restrict to only the useful information. 
merged_global = merged_global[['IPTGuM', 'mutant', 'repressors', 'fold_change']]

idx = {'Q21M':1, 'Q21A':2, 'Y20I':3}
for k in idx.keys():
    merged_global.loc[merged_global['mutant']==k, 'id'] = int(idx[k])
    
merged_global = merged_global[(merged_global['fold_change'] >= -0.2) & (merged_global['fold_change'] <= 1.3)]

In [45]:
# Assemble the data frame for fitting each individual strain.
unique_R = merged_global.repressors.unique()
unique_mut = merged_global.mutant.unique()
n_combinations = len(unique_R) * len(unique_mut)
label_key = {}
label = 1
for i, R in enumerate(unique_R):
    for j, m in enumerate(unique_mut):
        merged_global.loc[(merged_global['mutant'] == m) & (merged_global['repressors']== R), 
                         'fit_idx'] = label
        label_key[label] = (R, m)
        label += 1 

In [46]:
# Compile the stan model.
model_code = assemble_StanModelCode('../stan/heirarchical_single_strain_fit.stan',
                                   '../stan/functions.stan')
sm = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ecef4474642aa89221d27f86523ebb8d NOW.


In file included from /var/folders/2q/lvh2zsws3lxckq8xtkn_84z80000gn/T/tmpfq15qwxp/stanfit4anon_model_ecef4474642aa89221d27f86523ebb8d_2973807668323558287.cpp:599:
In file included from /anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/ndarrayobject.h:18:
In file included from /anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/ndarraytypes.h:1816:
/anaconda3/lib/python3.6/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: "Using deprecated NumPy API, disable it by "          "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it by " \
 ^
/var/folders/2q/lvh2zsws3lxckq8xtkn_84z80000gn/T/tmpfq15qwxp/stanfit4anon_model_ecef4474642aa89221d27f86523ebb8d_2973807668323558287.cpp:9155:30: warning: comparison of integers of different signs: 'Py_ssize_t' (aka 'long') and 'std::_

In [47]:
# Assemble the data dictionary.
data_dict = {'J': int(n_combinations), 'N': len(merged_global), 'trial':merged_global['fit_idx'].astype(int),
            'R':merged_global['repressors'], 'n_ns':4.6E6, 'ep_ai':4.5, 'n_sites':int(2), 
             'c': merged_global['IPTGuM'], 'fc':merged_global['fold_change']}

# Sample!
samples = sm.sampling(data=data_dict, iter=3000, chains=1)

/anaconda3/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [48]:
# Convert to dataframe and rename columns. 
strain_fit_df = chains_to_dataframe(samples)
new_cols = {'{}.{}'.format(v.split('.')[0], i+1):'R_{}_{}_{}'.format(int(label_key[k][0]), v.split('.')[0], label_key[k][1]) for i, k in enumerate(label_key.keys()) for v in samples.constrained_param_names()}
strain_fit_df.rename(columns = new_cols, inplace=True)

In [49]:
strain_fit_stats = mut.stats.compute_statistics(strain_fit_df)
strain_fit_stats

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


,parameter,mode,hpd_min,hpd_max
0,R_1220_ep_R_Q21M,-17.586200,-19.486747,-12.351102
1,R_60_ep_R_Q21M,-17.279181,-17.698844,-14.051595
2,R_60_ep_R_Q21A,-9.780386,-9.840180,-9.355765
3,R_60_ep_R_Y20I,-9.906897,-10.053868,-9.657171
4,R_1220_ep_R_Q21A,-10.237569,-10.583028,-10.150342
5,R_1220_ep_R_Y20I,-9.806535,-10.315033,-9.447269
6,R_124_ep_R_Q21M,-16.575739,-18.935257,-13.369169
7,R_124_ep_R_Q21A,-10.116932,-10.180570,-9.920576
8,R_124_ep_R_Y20I,-9.464715,-9.710105,-9.219354
9,R_260_ep_R_Q21M,-15.545933,-19.101386,-12.692969


In [50]:
strain_fit_df

,R_1220_ep_R_Q21M,R_60_ep_R_Q21M,R_60_ep_R_Q21A,R_60_ep_R_Y20I,R_1220_ep_R_Q21A,R_1220_ep_R_Y20I,R_124_ep_R_Q21M,R_124_ep_R_Q21A,R_124_ep_R_Y20I,R_260_ep_R_Q21M,...,R_60_sigma_Q21A,R_60_sigma_Y20I,R_1220_sigma_Q21A,R_1220_sigma_Y20I,R_124_sigma_Q21M,R_124_sigma_Q21A,R_124_sigma_Y20I,R_260_sigma_Q21M,R_260_sigma_Q21A,R_260_sigma_Y20I
0,-17.601382,-14.442401,-9.498585,-9.961036,-10.386872,-9.676231,-15.480639,-10.070216,-9.640551,-15.153525,...,0.053816,0.045272,0.035865,0.103541,0.095410,0.053975,0.081246,0.070225,0.107558,0.111535
1,-15.439385,-14.451878,-9.655198,-9.807939,-10.272059,-9.797098,-15.167296,-9.997192,-9.388305,-13.713783,...,0.058791,0.050346,0.042148,0.101176,0.087747,0.050478,0.086087,0.084002,0.082332,0.111447
2,-13.079783,-14.784646,-9.647110,-9.775254,-10.244456,-9.765040,-17.378842,-9.937302,-9.235239,-13.250792,...,0.062760,0.041374,0.038539,0.131418,0.089860,0.047266,0.077089,0.084359,0.086632,0.110470
3,-12.952565,-14.902568,-9.501378,-10.071529,-10.242759,-9.651648,-16.201845,-9.997637,-9.389719,-14.986215,...,0.055415,0.044225,0.041437,0.122941,0.081030,0.050025,0.087042,0.080398,0.084093,0.119533
4,-16.562055,-15.004336,-9.721577,-9.968458,-10.443675,-9.786228,-14.662368,-10.122999,-9.271627,-14.744958,...,0.062543,0.046795,0.046832,0.129096,0.084431,0.047608,0.071363,0.088676,0.086379,0.105762
5,-14.794832,-15.078304,-9.560676,-9.873701,-10.206656,-10.169027,-17.958905,-10.023151,-9.517890,-13.724620,...,0.058568,0.045633,0.032897,0.100792,0.087871,0.049823,0.068093,0.086040,0.078800,0.112517
6,-15.912885,-14.616580,-9.623343,-9.879495,-10.201885,-10.133049,-18.569490,-10.063487,-9.548105,-13.303492,...,0.054688,0.043412,0.035401,0.101760,0.069774,0.047063,0.080848,0.069767,0.075457,0.119861
7,-13.684104,-15.391020,-9.727432,-9.887966,-10.488121,-9.619196,-19.313576,-10.105991,-9.768433,-14.227570,...,0.054272,0.051062,0.040367,0.112229,0.081700,0.052983,0.092895,0.080044,0.082174,0.099399
8,-13.372373,-14.947927,-9.589876,-9.812836,-10.362149,-9.670823,-14.585338,-9.983298,-9.626084,-14.662847,...,0.063260,0.046382,0.037016,0.106926,0.071647,0.059128,0.073415,0.094451,0.075630,0.094022
9,-13.717004,-18.706108,-9.853570,-9.935811,-10.484609,-9.672841,-14.473244,-10.119014,-9.638070,-14.238140,...,0.060863,0.043367,0.043309,0.127417,0.088678,0.048414,0.071406,0.067509,0.082416,0.121446


In [52]:
import imp
imp.reload(mut.thermo)

<module 'mut.thermo' from '../../mut/thermo.py'>

In [53]:
# Compute  and plot the properties. 
ka = 139
ki = 0.53

p1 = bokeh.plotting.figure(plot_width=340, plot_height=300, x_axis_type='log',
                          y_axis_type='log', x_axis_label='repressors per cell',
                          y_axis_label='leakiness')

p2 = bokeh.plotting.figure(plot_width=340, plot_height=300, x_axis_type='log',
                           x_axis_label='repressors per cell',
                          y_axis_label='saturation')

p3 = bokeh.plotting.figure(plot_width=340, plot_height=300, x_axis_type='log',
                          x_axis_label='repressors per cell',
                          y_axis_label='dynamic range')

p4 = bokeh.plotting.figure(plot_width=340, plot_height=300, x_axis_type='log',
                          x_axis_label='repressors per cell', y_axis_type='log',
                          y_axis_label='ec50')

p5 = bokeh.plotting.figure(plot_width=340, plot_height=300, x_axis_type='log',
                          x_axis_label='repressors per cell',
                          y_axis_label='effective hill coefficient')
canvas = [p1, p2, p3, p4, p5]
rep_range = np.logspace(0, 4, 500)
color_palette = bokeh.palettes.viridis(5)

for i, m in enumerate(ep_R):
 
    # mesh the bounds and instantiate the architecture.
    low = ep_R_low[i]
    high = ep_R_high[i]
    vals = np.array([m, low, high])
    r, ep = np.meshgrid(rep_range, vals)
    
    arch = mut.thermo.SimpleRepression(R=r, ep_r=ep, ka=ka, ki=ki, ep_ai=4.5,
                                      effector_conc=0, n_sites=2) 
    properties = arch.compute_properties()
    for j, p in enumerate(properties.keys()):
        canvas[j].line(rep_range, properties[p][0, :], color=color_palette[i])
        bandx = np.append(rep_range, rep_range[::-1])
        bandy = np.append(properties[p][1, :], properties[p][2, ::-1])
        canvas[j].patch(bandx, bandy, color=color_palette[i], alpha=0.4)
    

# Plot the data.
for i, m in enumerate(muts):
    _d = merged_global[merged_global['mutant']==m]
    
    # Plot the leakiness.
    grouped = _d.groupby(['repressors', 'IPTGuM'])
    leak, sat = {}, {}
    for g, d in grouped:
        mean_fc = d['fold_change'].mean()
        sem_fc = d['fold_change'].std() / np.sqrt(len(d))
        if g[1] == 0.0:     
            p1.circle(g[0], mean_fc, color=color_palette[i], legend=m)
            p1.line((g[0], g[0]), (mean_fc - sem_fc, mean_fc + sem_fc),
                        color=color_palette[i], line_width=2)
            leak[g[0]] = (mean_fc, sem_fc)
        if g[1] == 5000.0:
            p2.circle(g[0], mean_fc, color=color_palette[i])
            p2.line((g[0], g[0]), (mean_fc - sem_fc, mean_fc + sem_fc),
                        color=color_palette[i], line_width=2)           
            sat[g[0]] = (mean_fc, sem_fc)
    for j, r in enumerate(leak.keys()):
        p3.circle(r, sat[r][0] - leak[r][0], color=color_palette[i])
        p3.line((r, r), ((sat[r][0] - sat[r][1]) - (leak[r][0] + leak[r][1]),
                         (sat[r][0] + sat[r][1]) - (leak[r][0] - leak[r][1])),
               color=color_palette[i], line_width=2)
        
# Plot the EC50 and Effective hill.
for i, R in enumerate(unique_R):
    for j, M in enumerate(muts):
        # Isolate the statistics for each.
        ep = strain_fit_stats.loc[strain_fit_stats['parameter']=='R_{}_ep_R_{}'.format(int(R), M)]['mode']
        fit_ka = strain_fit_stats.loc[strain_fit_stats['parameter']=='R_{}_ka_{}'.format(int(R), M)]['mode']
        fit_ki = strain_fit_stats.loc[strain_fit_stats['parameter']=='R_{}_ki_{}'.format(int(R), M)]['mode']
       
        # Instantiate the architecture and compute the properties
        arch = mut.thermo.SimpleRepression(R=R, ep_r=ep.values[0], ka=fit_ka.values[0], ki=fit_ki.values[0], effector_conc=0, 
                                          n_ns=4.6E6, ep_ai=4.5, n_sites=2)
        ec50 = arch.ec50()
        hill = arch.effective_hill()
        
        # Compute the credible region for each.
        traces = strain_fit_df[['R_{}_ep_R_{}'.format(int(R), M),
                                'R_{}_ka_{}'.format(int(R), M),
                                'R_{}_ki_{}'.format(int(R), M)]]
        labels = traces.keys()
        arch = mut.thermo.SimpleRepression(R=R, ep_r=traces[labels[0]], ka=traces[labels[1]], ki=traces[labels[2]], effector_conc=0, 
                                          n_ns=4.6E6, ep_ai=4.5, n_sites=2)
        ec50_bounds = mut.stats.compute_hpd(arch.ec50(), mass_frac=0.95)
        hill_bounds = mut.stats.compute_hpd(arch.effective_hill(), mass_frac=0.95)
        p4.square(R, ec50, color=color_palette[j])
        p4.line((R, R), (ec50_bounds[0], ec50_bounds[1]), line_width=2, color=color_palette[j])
        p5.line((R, R), (hill_bounds[0], hill_bounds[1]), line_width=2, color=color_palette[j])
        p5.square(R, hill, color=color_palette[j])
        
p1.legend.location = 'bottom_left' 
layout = bokeh.layouts.gridplot([[p1, p2], [p3, p4], [p5]])
bokeh.io.show(layout)

In [ ]:
# Assemble the stan model. 
model_code = assemble_StanModelCode('../stan/heirarchical_global_fit.stan', '../stan/functions.stan')
sm = pystan.StanModel(model_code=model_code)

In [221]:
# Assemble the data dictionary.
data_dict = {'J': 3, 'N': len(merged_global), 'R':merged_global['repressors'],
            'c':merged_global['IPTGuM'], 'trial':merged_global['id'].astype(int), 'n_ns':4.6E6,
            'ep_ai':4.5, 'n_sites':2, 'fc':merged_global['fold_change']}
global_fit = sm.sampling(data=data_dict, iter=4000, chains=2)

/anaconda3/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [222]:
global_df = chains_to_dataframe(global_fit)
global_fit_stats = mut.stats.compute_statistics(global_df)

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


In [223]:
global_fit_stats

,parameter,mode,hpd_min,hpd_max
0,ep_R.1,-17.602177,-18.113386,-14.044916
1,ep_R.2,-10.216548,-10.362980,-10.171308
2,ep_R.3,-9.722705,-9.895033,-9.634323
3,ka.1,124.711990,139.807561,178.097983
4,ka.2,107.444829,84.347232,191.623039
5,ka.3,42.212189,30.053284,69.369623
6,ka_tilde.1,4.826007,4.949202,5.190643
7,ka_tilde.2,4.676977,4.468650,5.282614
8,ka_tilde.3,3.742709,3.439081,4.272151
9,ki.1,0.175929,0.044180,1.061920
